# Amplifierd API - Sessions & Messages

This notebook demonstrates session management and messaging capabilities.

## Overview

Sessions represent conversational contexts for LLM interactions. Each session:
- Has a unique ID
- Uses a specific profile
- Maintains a message transcript
- Tracks conversation context

## Setup

Import libraries and configure the client:

In [1]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    """Print formatted HTTP response."""
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            print(json.dumps(response.json(), indent=2))
        except json.JSONDecodeError:
            print(response.text)


print("✓ Setup complete")

✓ Setup complete


## Creating a Session

Create a new session with a profile and optional context:

In [4]:
# Create a session with default profile
create_data = {"profile": "default", "context": {"task": "API testing", "user": "developer"}}

response = requests.post(f"{API_BASE}/sessions", json=create_data)
print_response(response, "CREATE SESSION")

# Save session ID for later use
if response.ok:
    session_data = response.json()
    session_id = session_data["id"]
    print(f"\n✓ Created session: {session_id}")
else:
    session_id = None
    print("\n✗ Failed to create session")


CREATE SESSION
Status: 201 Created
{
  "id": "c4f27d83-2c6e-4955-9724-61459cdd874b",
  "profile": "default",
  "context": {
    "task": "API testing",
    "user": "developer"
  },
  "created_at": "2025-11-21T00:17:22.146700Z",
  "updated_at": "2025-11-21T00:17:22.146714Z",
  "message_count": 0
}

✓ Created session: c4f27d83-2c6e-4955-9724-61459cdd874b


## Listing Sessions

Get all available sessions:

In [5]:
response = requests.get(f"{API_BASE}/sessions")
print_response(response, "LIST SESSIONS")

if response.ok:
    sessions = response.json()
    print(f"\n✓ Found {len(sessions)} session(s)")


LIST SESSIONS
Status: 200 OK
[
  {
    "id": "c4f27d83-2c6e-4955-9724-61459cdd874b",
    "profile": "default",
    "created_at": "2025-11-21T00:17:22.146700Z",
    "updated_at": "2025-11-21T00:17:22.146714Z",
    "message_count": 0
  },
  {
    "id": "63c050b6-ac34-4536-8be2-124d85916c80",
    "profile": "default",
    "created_at": "2025-11-21T00:16:45.184900Z",
    "updated_at": "2025-11-21T00:16:45.184909Z",
    "message_count": 0
  }
]

✓ Found 2 session(s)


## Getting Session Details

Retrieve information about a specific session:

In [6]:
if session_id:
    response = requests.get(f"{API_BASE}/sessions/{session_id}")
    print_response(response, f"GET SESSION: {session_id}")
else:
    print("No session ID available. Create a session first.")


GET SESSION: c4f27d83-2c6e-4955-9724-61459cdd874b
Status: 200 OK
{
  "id": "c4f27d83-2c6e-4955-9724-61459cdd874b",
  "profile": "default",
  "context": {
    "task": "API testing",
    "user": "developer"
  },
  "created_at": "2025-11-21T00:17:22.146700Z",
  "updated_at": "2025-11-21T00:17:22.146714Z",
  "message_count": 0
}


## Sending Messages

Send a message to a session:

In [7]:
if session_id:
    message_data = {
        "content": "Hello, amplifierd! This is a test message.",
        "metadata": {"source": "jupyter_notebook", "timestamp": "2025-01-01T00:00:00Z"},
    }

    response = requests.post(f"{API_BASE}/sessions/{session_id}/messages", json=message_data)
    print_response(response, "SEND MESSAGE")
else:
    print("No session ID available. Create a session first.")


SEND MESSAGE
Status: 201 Created
{
  "role": "user",
  "content": "Hello, amplifierd! This is a test message.",
  "timestamp": "2025-11-21T00:17:55.954323Z",
  "metadata": {}
}


## Retrieving Messages

Get the message transcript for a session:

In [8]:
if session_id:
    response = requests.get(f"{API_BASE}/sessions/{session_id}/messages")
    print_response(response, "GET MESSAGES")

    if response.ok:
        messages = response.json()
        print(f"\n✓ Retrieved {len(messages)} message(s)")
        for msg in messages:
            print(f"  - {msg['role']}: {msg['content'][:50]}...")
else:
    print("No session ID available. Create a session first.")


GET MESSAGES
Status: 200 OK
{
  "session_id": "c4f27d83-2c6e-4955-9724-61459cdd874b",
  "messages": [
    {
      "role": "user",
      "content": "Hello, amplifierd! This is a test message.",
      "timestamp": "2025-11-21T00:17:55.954323Z",
      "metadata": {}
    }
  ]
}

✓ Retrieved 2 message(s)


TypeError: string indices must be integers, not 'str'

## Sending Multiple Messages

Simulate a conversation:

In [ ]:
if session_id:
    messages_to_send = [
        {"content": "What are amplifier profiles?"},
        {"content": "How do I activate a profile?"},
        {"content": "Can I list all available modules?"},
    ]

    for msg_data in messages_to_send:
        response = requests.post(f"{API_BASE}/sessions/{session_id}/messages", json=msg_data)
        if response.ok:
            print(f"✓ Sent: {msg_data['content']}")
        else:
            print(f"✗ Failed: {msg_data['content']}")

    # Get updated transcript
    response = requests.get(f"{API_BASE}/sessions/{session_id}/messages")
    if response.ok:
        messages = response.json()
        print(f"\n✓ Total messages in session: {len(messages)}")
else:
    print("No session ID available. Create a session first.")

## Resuming a Session

Resume an existing session:

In [ ]:
if session_id:
    response = requests.post(f"{API_BASE}/sessions/{session_id}/resume")
    print_response(response, f"RESUME SESSION: {session_id}")
else:
    print("No session ID available. Create a session first.")

## Error Handling: Non-Existent Session

Attempting to access a session that doesn't exist:

In [ ]:
fake_session_id = "non-existent-session-id"
response = requests.get(f"{API_BASE}/sessions/{fake_session_id}")
print_response(response, "GET NON-EXISTENT SESSION")

if response.status_code == 404:
    print("\n✓ Correctly returns 404 for non-existent session")

## Deleting a Session

Clean up by deleting the session:

In [ ]:
if session_id:
    response = requests.delete(f"{API_BASE}/sessions/{session_id}")
    print_response(response, f"DELETE SESSION: {session_id}")

    if response.status_code == 204:
        print("\n✓ Session deleted successfully")

        # Verify deletion
        response = requests.get(f"{API_BASE}/sessions/{session_id}")
        if response.status_code == 404:
            print("✓ Confirmed session no longer exists")
else:
    print("No session ID available.")

## Complete Session Lifecycle Example

Full workflow from creation to deletion:

In [ ]:
def session_lifecycle_example():
    """Demonstrate complete session lifecycle."""

    # 1. Create session
    print("1. Creating session...")
    response = requests.post(f"{API_BASE}/sessions", json={"profile": "default"})
    if not response.ok:
        print("✗ Failed to create session")
        return

    session_id = response.json()["session_id"]
    print(f"✓ Created: {session_id}")

    # 2. Send messages
    print("\n2. Sending messages...")
    for i in range(3):
        response = requests.post(f"{API_BASE}/sessions/{session_id}/messages", json={"content": f"Message {i + 1}"})
        if response.ok:
            print(f"✓ Sent message {i + 1}")

    # 3. Retrieve transcript
    print("\n3. Retrieving transcript...")
    response = requests.get(f"{API_BASE}/sessions/{session_id}/messages")
    if response.ok:
        messages = response.json()
        print(f"✓ Retrieved {len(messages)} messages")

    # 4. Delete session
    print("\n4. Deleting session...")
    response = requests.delete(f"{API_BASE}/sessions/{session_id}")
    if response.status_code == 204:
        print("✓ Deleted successfully")

    print("\n✓ Lifecycle complete")


session_lifecycle_example()

## Summary

You've learned:
- ✓ Creating sessions with profiles and context
- ✓ Listing and retrieving session details
- ✓ Sending and retrieving messages
- ✓ Resuming existing sessions
- ✓ Deleting sessions
- ✓ Complete session lifecycle management

## API Endpoints Reference

| Method | Endpoint | Description |
|--------|----------|-------------|
| POST | `/api/v1/sessions` | Create new session |
| GET | `/api/v1/sessions` | List all sessions |
| GET | `/api/v1/sessions/{id}` | Get session details |
| POST | `/api/v1/sessions/{id}/resume` | Resume session |
| DELETE | `/api/v1/sessions/{id}` | Delete session |
| POST | `/api/v1/sessions/{id}/messages` | Send message |
| GET | `/api/v1/sessions/{id}/messages` | Get messages |

## Next Steps

Continue to configuration management notebooks:
- **03-profile-management.ipynb** - Profile operations
- **04-collection-management.ipynb** - Collection management
- **05-module-management.ipynb** - Module configuration